### Preamble

In [50]:
import $ivy.`org.scalatest::scalatest:3.2.16`
import org.scalatest.{Filter => _, _}, flatspec._, matchers._

import $ivy.$                                

import org.scalatest.{Filter => _, _}, flatspec._, matchers._


# Topic 6. Higher-order functions and modular programming

In this notebook, we will see that higher-order functions (HOFs) are essential modularity devices, and we will introduce the most common higher-order functions that operate over many different data structures.

### References

[__Programming in Scala, 
A comprehensive step-by-step guide__](https://www.artima.com/shop/programming_in_scala_3ed) Third Edition.
by Martin Odersky, Lex Spoon, and Bill Venners. 

- Chapter 6. Functional objects
- Chapter 8. Functions and closures
- Chapter 16. Working with Lists
- Chapter 23. For Expressions Revisited

[__Functional programming in Scala__](https://www.manning.com/books/functional-programming-in-scala), by Paul Chiusano and Runar Bjarnason.

- Chapter 3. Functional data structures

[__Functional programming simplified__](https://alvinalexander.com/downloads/fpsimplified-free-preview.pdf), by Alvin Alexander.

- Chapter 22. Functions Are Variables, Too
- Chapter 23. Using Methods As If They Were Functions
- Chapter 24. How to Write Functions That Take Functions as Input Parameters
- Chapter 25. How to Write a ‘map’ Function
- Chapter 27. Functions Can Have Multiple Parameter Groups
- Chapter 28. Partially-Applied Functions (and Currying)

## `FoldRight`: divide and conquer

HOFs shine when the time comes to break monoliths. For instance, let's consider the following two functions:

In [2]:
// sum of elements
def sum(l: List[Int]): Int = 
    l match 
        case Nil => ??? : Int
        case head :: tail => 
            val tailSol: Int = sum(tail)
            ??? : Int

defined function sum

In [ ]:
// reverse
def reverse[A](l: List[A]): List[A] = 
    l match 
        case Nil => ??? : List[A]
        case head :: tail => 
            val tailSol: List[A] = reverse(tail)
            ??? : List[A]

In [2]:
// sum of elements
def sum(l: List[Int]): Int = 
    l match 
        case Nil => 0 : Int
        case head :: tail => 
            val tailSol: Int = sum(tail)
            head + tailSol : Int

defined function sum

In [3]:
// reverse
def reverse[A](l: List[A]): List[A] = 
    l match 
        case Nil => Nil : List[A]
        case head :: tail => 
            val tailSol: List[A] = reverse(tail)
            tailSol :+ head : List[A]

defined function reverse

In [4]:
def dyv[A, B](l: List[A])(nil: B, cons: (A, B) => B) : B = 
    l match 
        case List() => nil : B
        case head :: tail => 
            val tailSol: B = dyv(tail)(nil, cons)
            cons(head, tailSol) : B

defined function dyv

In [5]:
// reverse
def reverse[A](l: List[A]): List[A] = 
    dyv[A, List[A]](l)(
        Nil : List[A],
        (head: A, tailReverse: List[A]) => 
            tailReverse :+ head : List[A]
    )

defined function reverse

In [6]:
// sum of elements
def sum(l: List[Int]): Int = 
    dyv[Int, Int](l)(
        0 : Int,
        (head: Int, tailSum: Int) => 
            head + tailSum : Int
    )

defined function sum

In [8]:
for (i <- 1 to 10)
    println(i)

1
2
3
4
5
6
7
8
9
10


In [15]:
def multiply(l: List[Int]): Int = 
    l.foldRight(1 : Int): (head, tailMultiplied: Int) => 
        head * tailMultiplied

defined function multiply

In [16]:
def multiply(l: List[Int]): Int = 
    l.foldRight(1)(_*_)

defined function multiply

In [17]:
1 :: 2 :: 3 :: 4 :: 5 :: Nil
1 *  2 *  3 *  4 *  5 *  1
           

res17_0: List[Int] = List(1, 2, 3, 4, 5)
res17_1: Int = 120

In [9]:
def dyv[A, B](l: List[A])(nil: B)(cons: (A, B) => B) : B = 
    l match 
        case List() => nil : B
        case head :: tail => 
            val tailSol: B = dyv(tail)(nil)(cons)
            cons(head, tailSol) : B

defined function dyv

In [10]:
// sum of elements
def sum(l: List[Int]): Int = 
    dyv(l)(0 : Int): (head, tailSum) => 
        head + tailSum : Int

defined function sum

In [11]:
// sum of elements
def sum(l: List[Int]): Int = 
    dyv(l)(0 : Int): 
        (head, tailSum) => head + tailSum : Int

defined function sum

In [12]:
// sum of elements
def sum(l: List[Int]): Int = 
    dyv(l)(0 : Int)( 
        (head, tailSum) => head + tailSum : Int
    )

defined function sum

In [ ]:
reverse(List(1,2,3,4,5)) == List(5,4,3,2,1)
reverse(1 :: List(2,3,4,5)) == List(5,4,3,2) :+ 1

In [ ]:
// product of elements 

These functions clearly share a common logic: their only differences are the value which is returned when the list is empty, and the function used to combine numbers (`+` and `*`, respectively). We can abstract away these differences and arrive to a more generic function which encodes that common logic:

In [ ]:
// monomorphic `combine` method

which allows us to re-define in a modular way the `sum` and `multiply` functions:

In [ ]:
def sum(list: List[Int]): Int = 
    ???

In [ ]:
def multiply(list: List[Int]): Int = 
    ???

But we don't need to constrain ourselves to integers. In its generic version, the `combine` function is actually the `foldRight` higher-order function (for `List`s):

In [ ]:
// more generically: foldRight


Graphically, we can explain the behaviour of `foldRight` as follows: 

![with elements](../../images/foldRight.1.svg)

(all images credit to [Scala Visual Reference](https://superruzafa.github.io/visual-scala-reference/))

We can understand `foldRight` as an implementation of the divide-and-conquer design pattern: first, divide your problems in subproblems; second, solve these sub-problems; last, compose their solutions to obtain the solution to the overall problem. If the sub-problems are simple enough they can be solved directly; otherwise, they are solved recursively. In the case of lists:
- The problem is obtaining a value of type `B` for a given list.
- The only sub-problem corresponds to the tail of that list. 
- The arguments of the `foldRight` function tell us how to obtain the solution for the empty list (the atomic problem which can not be further decomposed), and how to obtain the solution from the solution to the sub-problem.

From this perspective, the implementation of `sum` and `multiply` is exactly the same as before when we used the function `combine`, although we can explain them differently:

In [ ]:
def sum(list: List[Int]): Int = 
    foldRight[Int, Int](list)(
        0, // direct solution to atomic problem
        (head, subsol) => head + subsol // composing sub-solution
    )

In [ ]:
def multiply(list: List[Int]): Int = 
    foldRight(list)(
        1, // direct solution
        (head: Int, subsol: Int) => head * subsol // compose
    )

#### HOFs in the Scala API

The `foldRight` HOF, as all the HOFs that we will explain in this notebook, are actually part of the standard Collections library of Scala. Typically, they are implemented as member methods of the corresponding collection class. For lists, the [Scala API](https://www.scala-lang.org/api/2.13.1/scala/collection/immutable/List.html) offers something like this:

In [ ]:
object Standard{
    class List[A]{
        def foldRight[B](b: B)(f: (B, A) => B): B = ???
    }
}

So, invocations of the standard `foldRight` do not receive the list as argument, but are common method invocations on some `List` instance:

In [ ]:
// foldRight over lists



Normally, we will use standard HOFs instead of our home-made versions, once they are explained. 

## The Hall of Fame of HOFs

Besides `compose`, `andThen` and `foldRight`, there are other famous higher-order functions which work great as modularity devices: 

In [ ]:
def foldLeft[A, B](list: List[A])(b: B)(f: (B, A) => B): B = ???
def filter[A](list: List[A])(f: A => Boolean): List[A] = ???
def map[A, B](list: List[A])(f: A => B): List[B] = ???
def flatMap[A, B](list: List[A])(f: A => List[B]): List[B] = ???

But most of these functions do not only work for `List`, they work also for `Option`, `Either[A, ?]`, and many other data structures. For instance:

In [ ]:
def fold[A, B](opt: Option[A])(none: B)(some: A => B): B = ???
def filter[A](opt: Option[A])(f: A => Boolean): Option[A] = ???
def map[A, B](opt: Option[A])(f: A => B): Option[B] = ???  
def flatMap[A, B](opt: Option[A])(f: A => Option[B] ): Option[B] = ???

In [ ]:
def fold[A, B, C](opt: Either[A, B])(left: A => C, right: B => C): C = ???
def filter[A, B](opt: Either[A, B])(f: B => Boolean): Either[A, B] = ???
def map[A, B, C](opt: Either[A, B])(f: B => C): Either[A, C] = ???  
def flatMap[A, B, C](opt: Either[A, B])(f: B => Either[A, C] ): Either[A, C] = ???

The `fold` function is the catamorphism for `Option` and `Either[A, ?]`, in the same way than `foldRight` is the catamorphism for lists (unfortunately, the concept of catamorphism won't be explained due to lack of time). 

## FoldLeft: a better loop

The `foldLeft` HOF is the functional way to implement common imperative algorithms. For instance:

In [ ]:
// length function, imperatively

def member[A](l: List[A], e: A): Boolean = 
    def step(out: Boolean, aux: List[A]): Boolean = 
        aux match 
            case Nil => out
            case head :: tail => 
                step(???(out, head), tail)

    step(???, l)

In [ ]:
// reverse function, imperatively
def reverse[A](l: List[A]): List[A] = 
    def step(out: List[A], aux: List[A]): List[A] = 
        aux match 
            case Nil => out
            case head :: tail => 
                step(???(out, head), tail)

    step(???, l)

In [19]:
// length function, imperatively

def member[A](l: List[A], e: A): Boolean = 
    def step(out: Boolean, aux: List[A]): Boolean = 
        aux match 
            case Nil => out
            case head :: tail => 
                step(out || head == e, tail)

    step(false, l)

defined function member

In [20]:
// reverse function, imperatively
def reverse[A](l: List[A]): List[A] = 
    def step(out: List[A], aux: List[A]): List[A] = 
        aux match 
            case Nil => out
            case head :: tail => 
                step(head :: out, tail)

    step(Nil, l)

defined function reverse

In [20]:
// reverse function, imperatively
def reverse[A](l: List[A]): List[A] = 
    def step(out: List[A], aux: List[A]): List[A] = 
        aux match 
            case Nil => out
            case head :: tail => 
                step(head :: out, tail)

    step(Nil, l)

defined function reverse

In [12]:
// 
def iterate[A, B](l: List[A])(initial: B)(update: (B, A) => B): B = 
    var out: B = initial
    for (e <- l)
        out = update(out, e)
    out

defined function iterate

In [3]:
// 
def iterate[A, B](l: List[A])(initial: B)(update: (B, A) => B): B = 
    def step(out: B, aux: List[A]): B = 
        aux match 
            case Nil => out
            case head :: tail => 
                step(update(out, head), tail)

    step(initial, l)

defined function iterate

In [11]:
@annotation.tailrec
def iterate[A, B](l: List[A])(initial: B)(update: (B, A) => B): B = 
    l match 
        case Nil => initial
        case head :: tail => 
            iterate(tail)(update(initial, head))(update)

defined function iterate

In [5]:
// length function, imperatively

def member[A](l: List[A], e: A): Boolean = 
    iterate[A, Boolean](l)(false): (out, head) => 
        out || head == e

defined function member

In [6]:
// length function, imperatively

def member[A](l: List[A], e: A): Boolean = 
    iterate(l)(false): (out, head) => 
        out || head == e

defined function member

In [9]:
val l: List[Int] = Nil
val l1: List[Char] = Nil
val l2: List[String] = Nil

l: List[Int] = List()
l1: List[Char] = List()
l2: List[String] = List()

In [7]:
// reverse function, imperatively
def reverse[A](l: List[A]): List[A] = 
    iterate(l)(Nil: List[A]): (out, head) => 
        head :: out

defined function reverse

In [8]:
// reverse function, imperatively
def reverse[A](l: List[A]): List[A] = 
    iterate[A, List[A]](l)(Nil): (out, head) => 
        head :: out

defined function reverse

In [22]:
// length function, imperatively

def member[A](l: List[A], e: A): Boolean = 
    l.foldLeft(false): (out, head) => 
        out || head == e

defined function member

In [21]:
// reverse function, imperatively
def reverse[A](l: List[A]): List[A] = 
    l.foldLeft(Nil: List[A]): (out, head) => 
        head :: out

defined function reverse

The following implementation abstracts away the differences in the above functions, and declares two additional parameters: the initial value of a mutable variable, and the update function executed in every step of the loop.

In [ ]:
// abstraction: foldLeft, imperatively


Similarly, the `foldLeft` function is typically used where a tail-recursive function is needed. Its recursive implementation is naturally tail-recursive:

In [ ]:
@annotation.tailrec
def foldLeft[A, B](list: List[A])(out: B)(update: (B, A) => B): B =
    ??? 

Using `foldLeft` we can implement functions at a higher-level of abstraction, i.e. using constructs which are nearer to the problem-domain, without caring about mutable variables, tail-safe recursion, etc. Its use also leads to very concise (and readable!) implementations. For instance, these are (almost) one-liner implementations of some of the functions of the last notebook on recursive functions:

In [ ]:
def length[A](list: List[A]): Int = 
    ??? 

In [ ]:
def sum(list: List[Int]): Int = 
    ???

In [ ]:
def reverse[A](list: List[A]): List[A] = 
    ??? 

But we may also use the `foldRight` function to implement them:

In [ ]:
def length[A](list: List[A]): Int = 
    ??? 

In [ ]:
def sum(list: List[Int]): Int = 
    ???

In [ ]:
def reverse[A](list: List[A]): List[A] = 
    ???

Functionally, both versions are equivalent, but the ones using `foldLeft` are more efficient. In particular, the `foldRight` version of `reverse` has cuadratic complexity. 

#### Implementing `foldRight` with `foldLeft`

The implementation that we gave for `foldRight` was not tail-recursive, so this will blow up the stack:

In [ ]:
// blowing up the stack 


In order to obtain a better implementation, we may first reverse the list and use `foldLeft` as follows: 

In [ ]:
def foldRightTR[A, B](list: List[A])(nil: B)(cons: (A, B) => B): B = 
    ???

This implementation works right with big lists:

In [ ]:
// check it out


#### Implementing `foldLeft` with `foldRight`

This is also possible, though challenging. We start from the tail-recursive implementation of `foldLeft`. First, we rearrange the arguments so that it now returns a function instead of a plain value of type `B`. 

In [ ]:
// foldLeftAux


This was in order to better expose the recursion pattern of `foldRight`. The solution is now obtained simply by noticing that `foldLeft(tail)(update)` is the solution for the tail:

In [ ]:
// foldLeftAux with foldRight


The original signature can then be implemented as follows: 

In [ ]:
def foldLeft[A, B](list: List[A])(out: B)(update: (B, A) => B): B =
    ???

## The `map` HOF

The `map` function is one of the essential HOFs. Its purpose is applying a function to the elements of a data structure, in such a way that the relationships between these elements do not change. The only thing that must be modified is the _content_ of the data structure, not its _shape_. This condition is expressed in the following equations:

1. `map(list)(identity) == list` for all `list: List[A]`
2. `map(map(list)(f))(g) == map(list)(g compose f)` for all `list: List[A]`, `f: A => B`, `g: B => C`

![map](../../images/map.svg)

The implementation for lists can be done as follows:

In [ ]:
isPositive(List(-1, 0, 1, -5)) == ??? :: List(false, true, false)

In [17]:
isPositive(List(-1, 0, 1, -5, -6, 2)) == List(false, false, true, false, false, true)
isPositive(List(-8, -2)) == List(false, false)

res17_0: Boolean = true
res17_1: Boolean = true

In [13]:
// map example 1
def isPositive(l: List[Int]): List[Boolean] = 
    l match 
        case Nil => ??? : List[Boolean]
        case head :: tail => 
            val tailSol: List[Boolean] = isPositive(tail)
            ??? : List[Boolean]

defined function isPositive

In [16]:
// map example 1
def isPositive(l: List[Int]): List[Boolean] = 
    l match 
        case Nil => Nil : List[Boolean]
        case head :: tail => 
            val tailSol: List[Boolean] = isPositive(tail)
            (head > 0) :: tailSol : List[Boolean]

defined function isPositive

In [18]:
// map example 1
def isPositive(l: List[Int]): List[Boolean] = 
    l.foldRight(??? : List[Boolean]): 
        (head, tailSol) => 
            ??? : List[Boolean]

defined function isPositive

In [19]:
// map example 1
def isPositive(l: List[Int]): List[Boolean] = 
    l.foldRight(Nil : List[Boolean]): 
        (head, tailSol) => 
            (head > 0) :: tailSol : List[Boolean]

defined function isPositive

In [21]:
Character.toUpperCase('a')
Character.toUpperCase('b')

res21_0: Char = 'A'
res21_1: Char = 'B'

In [25]:
toUpperCase(List('a', 'b', 'c', 'D')) == List('A', 'B', 'C', 'D')

res25: Boolean = true

In [23]:
// map example 2
def toUpperCase(l: List[Char]): List[Char] = 
    l match 
        case Nil => ??? : List[Char]
        case head :: tail => 
            val tailSol: List[Char] = toUpperCase(tail)
            ??? : List[Char]

defined function toUpperCase

In [22]:
// map example 2
def toUpperCase(l: List[Char]): List[Char] = 
    l.foldRight(??? : List[Char]): 
        (head, tailSol: List[Char]) => 
            ??? : List[Char]

defined function toUpperCase

In [24]:
// map example 2
def toUpperCase(l: List[Char]): List[Char] = 
    l.foldRight(Nil : List[Char]): 
        (head, tailSol: List[Char]) => 
            Character.toUpperCase(head) :: tailSol : List[Char]

defined function toUpperCase

In [25]:
// map example 2
def toUpperCase(l: List[Char]): List[Char] = 
    l.foldRight(Nil): 
        (head, tailSol) => 
            Character.toUpperCase(head) :: tailSol

-- [E007] Type Mismatch Error: cell26.sc:4:12 ----------------------------------
4 |            Character.toUpperCase(head) :: tailSol
  |            ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  |      Found:    List[Char]
  |      Required: scala.collection.immutable.Nil.type
  |
  |      The following import might make progress towards fixing the problem:
  |
  |        import sourcecode.Text.generate
  |
  |
  | longer explanation available when compiling with `-explain`
Compilation Failed

In [26]:
// map example 2
def toUpperCase(l: List[Char]): List[Char] = 
    l.foldRight(Nil: List[Char]): 
        (head, tailSol) => 
            Character.toUpperCase(head) :: tailSol

defined function toUpperCase

In [27]:
// map example 2
def toUpperCase(l: List[Char]): List[Char] = 
    l.foldRight[List[Char]](Nil): 
        (head, tailSol) => 
            Character.toUpperCase(head) :: tailSol

defined function toUpperCase

In [30]:
List[Int]()

res30: List[Int] = List()

In [28]:
// map example 2
def toUpperCase(l: List[Char]): List[Char] = 
    l.foldRight(List[Char]()): 
        (head, tailSol) => 
            Character.toUpperCase(head) :: tailSol

defined function toUpperCase

In [16]:
// map example 1
def isPositive(l: List[Int]): List[Boolean] = 
    l match 
        case Nil => Nil : List[Boolean]
        case head :: tail => 
            val tailSol: List[Boolean] = isPositive(tail)
            (head > 0) :: tailSol : List[Boolean]

defined function isPositive

In [32]:
// map example 1
def isPositive(l: List[Int]): List[Boolean] = 
    // l.map(head => head > 0)
    l.map(_ > 0)

defined function isPositive

In [31]:
// map example 2
def toUpperCase(l: List[Char]): List[Char] = 
    l match 
        case Nil => Nil : List[Char]
        case head :: tail => 
            val tailSol: List[Char] = toUpperCase(tail)
            Character.toUpperCase(head) :: tailSol : List[Char]

defined function toUpperCase

In [33]:
// map example 2
def toUpperCase(l: List[Char]): List[Char] = 
    // l.map(head => Character.toUpperCase(head))
    l.map(Character.toUpperCase)

defined function toUpperCase

In [34]:

def map[A, B](l: List[A])(f: A => B): List[B] = 
    l match 
        case Nil => Nil : List[B]
        case head :: tail => 
            val tailSol: List[B] = map(tail)(f)
            f(head) :: tailSol : List[B]

defined function map

In [36]:
map(List(1,-2,3,-5))(_ > 0)
map(List('a', 'b', 'c', 'd'))(Character.toUpperCase)

res36_0: List[Boolean] = List(true, false, true, false)
res36_1: List[Char] = List('A', 'B', 'C', 'D')

The `map` function is polymorphic in `A` and `B`, but we can't pass generic functions (aka polymorphic values) as parameters using `FunctionN`classes (these classes only wrap functions from specific types to specific result types). We may create polymorphic versions of `FunctionN` clases, but in order to keep things simple, we will define the test catalogue for `map` using a specific signature chosen at random (any other may serve as well):

In [ ]:
class TestMap(
    map: List[Int] => (Int => Boolean) => List[Boolean]
) extends AnyFlatSpec with should.Matchers:

    val isEven: Int => Boolean = _ % 2 == 0

    "mapping an empty list" should "return an empty list" in:
        map(List())(isEven) shouldBe ???

    "mapping an non-empty list" should "only change its content" in:
        map(List(1))(isEven) shouldBe ???
        map(List(1,2))(isEven) shouldBe ???
        map(List(1,2,3))(isEven) shouldBe ???

To run the tests we will pass the `map` function for `Ìnt` and `Boolean` types:

In [ ]:
run(TestMap(map[Int, Boolean]))

although it's not necessary to write the types explicitly (Scala will infer the types for us):

In [ ]:
run(TestMap(map))

The implementation of `map` is really close to the implementation of `foldRight`. Indeed, we can give a more modular implementation using this more basic HOF: 

In [ ]:
/*
def foldRight[A, B](list: List[A])(nil: B)(cons: (A, B) => B): B = 
    list match {
        case Nil => nil
        case head :: tail => cons(head, foldRight(tail)(nil)(cons))
    }
*/

def mapFR[A, B](list: List[A])(f: A => B): List[B] = 
    ???

In [ ]:
run(TestMap(mapFR))

### `map` for `Option`

The implementation of the `map` function given above works over lists, but we can also map optional values and many other data structures. The implementation for the data structure `Option` must satisfy the following test:

In [ ]:
class TestMapOption(
    map: Option[Int] => (Int => Boolean) => Option[Boolean]
) extends AnyFlatSpec with should.Matchers:
    
    val isEven: Int => Boolean = _ % 2 == 0

    "mapping the `None` value" should "return `None`" in:
        map(None)(isEven) shouldBe ???
    
    "mapping a `Some` value" should "only change its content" in:
        map(Option(3))(isEven) shouldBe ???
        map(Option(1))(isEven) shouldBe ???
        map(Option(2))(isEven) shouldBe ???

The implementation is simple:

In [ ]:
def mapOpt[A, B](maybeA: Option[A])(f: A => B): Option[B] = 
    ???

In [ ]:
run(TestMapOption(mapOpt))

## Filtering elements

Unlike `map`, the `filter` HOF allows us to change the shape of the data structure, removing those elements that do not satisfy a given predicate. 

![filter](../../images/filter.svg)

In [41]:
upperCase(List('a', 'b', 'C', 'A')) == List('C', 'A')

res41: Boolean = true

In [39]:
'A'.isUpper

res39: Boolean = true

In [37]:
def upperCase(l: List[Char]): List[Char] = 
    l.foldRight(??? : List[Char]): 
        (head, tailSol: List[Char]) => 
            ??? : List[Char]

defined function upperCase

In [40]:
def upperCase(l: List[Char]): List[Char] = 
    l.foldRight(Nil : List[Char]): 
        (head, tailSol: List[Char]) => 
            if head.isUpper then head :: tailSol : List[Char]
            else tailSol

defined function upperCase

In [42]:
def upperCase(l: List[Char]): List[Char] = 
    l.foldRight(Nil : List[Char]): 
        case (head, tailSol) if head.isUpper => head :: tailSol
        case (_, tailSol) => tailSol

defined function upperCase

In [44]:
positives(List(1,-2,-3,4)) == List(1,4)

res44: Boolean = true

In [43]:
def positives(l: List[Int]): List[Int] = 
    l.foldRight(Nil : List[Int]): 
        case (head, tailSol) if head > 0 => head :: tailSol
        case (_, tailSol) => tailSol

defined function positives

In [47]:
def positives(l: List[Int]): List[Int] = 
    // l.filter(head => head > 0)
    l.filter(_ > 0)

defined function positives

In [48]:
def upperCase(l: List[Char]): List[Char] = 
    l.filter(_.isUpper)

defined function upperCase

In [49]:
// filter, generically 
def filter[A](l: List[A])(pred: A => Boolean): List[A] = 
    l.foldRight(Nil : List[A]): 
        case (head, tailSol) if pred(head) => head :: tailSol
        case (_, tailSol) => tailSol

defined function filter

In [ ]:
class TestFilterList(
    filter: List[Int] => (Int => Boolean) => List[Int]
) extends AnyFlatSpec with should.Matchers:
    
    val isEven: Int => Boolean = _ % 2 == 0

    "filter" should "work" in:
        filter(List())(isEven) shouldBe ???
        filter(List(1))(isEven) shouldBe ???
        filter(List(1,3,5))(isEven) shouldBe ???
        filter(List(2,4,6))(isEven) shouldBe ???

In [ ]:
run(TestFilterList(filterR))

Using `foldRight` we can get a more modular implementation: 

In [ ]:
def filterFR[A](list: List[A])(predicate: A => Boolean): List[A] = 
    ???

In [ ]:
run(TestFilterList(filterFR))

### Filtering optional values

We only have a possible value, so implementing filtering is easy in this case:

In [ ]:
class TestFilterOption(
    filter: Option[Int] => (Int => Boolean) => Option[Int]
) extends AnyFlatSpec with should.Matchers:
    
    val isEven: Int => Boolean = _ % 2 == 0

    "filter" should "work" in:
        filter(None)(isEven) shouldBe ???
        filter(Some(1))(isEven) shouldBe ???
        filter(Some(2))(isEven) shouldBe ???

In [ ]:
def filter[A](maybeA: Option[A])(predicate: A => Boolean): Option[A] = 
    ???

In [ ]:
run(TestFilterOption(filter))

## FlatMapping data structures

Let's consider a paragraph represented as a list of sentences, where each sentence is in turn modelled as a string made of _words_ separated by blank spaces. We want to obtain a list of all the words in each sentence. 

In [51]:
class TestWords(
    words: List[String] => List[String]
) extends AnyFlatSpec with should.Matchers:
            
    val paragraph1 = List(
        "En un lugar",
        "de la Mancha", 
        "de cuyo nombre no",
        "quiero acordarme")
    
    "words" should "work" in:
        words(paragraph1) shouldBe 
            List("En", "un", "lugar", "de", "la", "Mancha", "de", "cuyo", "nombre", "no",
                 "quiero", "acordarme")

defined class TestWords

In [53]:
"no quiero acordarme".split(" ").toList

res53: List[String] = List("no", "quiero", "acordarme")

In [54]:
def word(paragraph: List[String]): List[String] = 
    var out: List[String] = Nil
    for (sentence <- paragraph)
        out = out ++ sentence.split(" ").toList
    out

defined function word

In [55]:
run(TestWords(word))

cell51$Helper$TestWords:
words
- should work


In [63]:
List(   "En un lugar",
        "de la Mancha", 
        "de cuyo nombre no",
        "quiero acordarme")
    .map(sentence => sentence.split(" ").toList)
    .flatten

res63: List[String] = List(
  "En",
  "un",
  "lugar",
  "de",
  "la",
  "Mancha",
  "de",
  "cuyo",
  "nombre",
  "no",
  "quiero",
  "acordarme"
)

In [64]:
List(   "En un lugar",
        "de la Mancha", 
        "de cuyo nombre no",
        "quiero acordarme")
    .flatMap(sentence => sentence.split(" ").toList)

res64: List[String] = List(
  "En",
  "un",
  "lugar",
  "de",
  "la",
  "Mancha",
  "de",
  "cuyo",
  "nombre",
  "no",
  "quiero",
  "acordarme"
)

In [58]:
def concatenate[A](l: List[List[A]]): List[A] = 
    l.foldRight(??? : List[A]): 
        (head: List[A], tailSol: List[A]) => 
            ??? : List[A]

defined function concatenate

In [60]:
concatenate(List(2,3) :: List(List(3,4), List(5,6), List(7))) == 
    List(2,3)++List(3,4,5,6,7)

res60: Boolean = true

In [61]:
List(List(2,3), List(4,5), List(1)).flatten

res61: List[Int] = List(2, 3, 4, 5, 1)

In [59]:
def concatenate[A](l: List[List[A]]): List[A] = 
    l.foldRight(Nil : List[A]): 
        (head: List[A], tailSol: List[A]) => 
            head ++ tailSol : List[A]

defined function concatenate

We may try to map the paragraph with a function that `split`s  each sentence into the words they are made of:

In [ ]:
val paragraph: List[String] = List("en un lugar", "de la Mancha")

???

but then we obtain a _list of lists_ of strings, not a list of plain strings. The solution is not far though: we have just to concatenate all the lists and we obtain what we need. The function `flatten` performs this concatenation:

In [ ]:
// flatten


The function `flatten` is actually provided by the Scala API. So, the implementation of the `words` function is as follows:

In [54]:
def word(paragraph: List[String]): List[String] = 
    var out: List[String] = Nil
    for (sentence <- paragraph)
        out = out ++ sentence.split(" ").toList
    out

defined function word

In [65]:
def words(paragraph: List[String]): List[String] = 
    paragraph.flatMap(_.split(" ").toList)

defined function words

In [66]:
run(TestWords(words))

cell51$Helper$TestWords:
words
- should work


This combination of the HOF `map` and the function `flatten` is so common, that it has been given a proper name: `flatMap`. 

In [ ]:
// flatMap 


![filter](../../images/flatMap.svg)

Using `flatMap` the word function is implemented even more easily:

In [ ]:
def words(paragraph: List[String]): List[String] = 
    ???

### FlatMapping optional values

In [ ]:
def flatMap[A, B](maybeA: Option[A])(cont: A => Option[B]): Option[B] = 
    ???

In [ ]:
class TestFlatMapOption(
    flatMap: Option[Int] => (Int => Option[Boolean]) => Option[Boolean]
) extends AnyFlatSpec with should.Matchers:
    
    def isPositiveEven(i: Int): Option[Boolean] = 
        if (i>=0) Some(i % 2 == 0)
        else None
    
    "flatMap" should "work" in:
        flatMap(None)(isPositiveEven) shouldBe ???
        flatMap(Some(5))(isPositiveEven) shouldBe ???
        flatMap(Some(-5))(isPositiveEven) shouldBe ???
        flatMap(Some(0))(isPositiveEven) shouldBe ???
        flatMap(Some(4))(isPositiveEven) shouldBe ???

In [ ]:
run(TestFlatMapOption(flatMap))

## All together: a modularity problem

As before, we start from a list of sentences, but now we want to take care of the possible extra spaces between each word. Also, we want to obtain not the words themselves but their lengths. So, the specification of our problem is a function `lengths`:

In [ ]:
class TestLengths(
    lengths: List[String] => List[Int]
) extends AnyFlatSpec with should.Matchers:
            
    val paragraph1 = List(
        "En un  lugar",
        "de  la Mancha ", 
        "de cuyo nombre no",
        "quiero        acordarme")
    
    "lengths" should "work" in:
        lengths(paragraph1) shouldBe 
            List(2, 2, 5, 2, 2, 6, 
                 2, 4, 6, 2, 6, 9)

We may try to implement this function imperatively using mutable variables:

In [ ]:
def lengthsM(paragraph: List[String]): List[Int] =
    ???

and this works, of course: 

In [ ]:
run(TestLengths(lengthsM))

but can we do it better? Yes, we can! Using HOFs we can get a more concise, understandable and reliable version, with a great level of reuse!

In [ ]:
def lengthsHOF(paragraph: List[String]): List[Int] = 
    ???

In [ ]:
run(TestLengths(lengthsHOF))

This solution contrasts with the mutable version in several respects:
- It's more **modular**, i.e. it's made of coarse-grained components: the HOFs `flatMap`, `filter` and `map`. The mutable version builds instead upon fine-grained components: `var`s, and `for` loops.
- The HOF components `flatMap`, etc., are actually very generic and domain-independent, and are typically part of standard libraries, so the level of **reuse** and **reliability** of the HOF-based solution is very high.
- The HOF-based solution is more **understandable**: it models the solution to the problem in terms of standard _transformations_ which are composed together using plain function composition. Moreover, these transformations are also at the right level of abstraction, i.e. it's natural to specify the solution to the problem in terms of flatMap, filter and map.

### For-comprehensions

The combination of `map`, `flatMap` and `filter` HOFs is so common, that Scala has a special syntax for them: for-comprehensions.

In [ ]:
def lengthsFC(paragraph: List[String]): List[Int] = 
    ???

In [ ]:
run(TestLengths(lengthsFC))